# Extracción, transformación y carga
### Detección automática de sarcasmos en textos utilizando Transformers
#### Ernesto Francisco Barrionuevo Aragonés 21727751

In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

Para la extraccion, se realizara en dos pasos. El primero para todas las frases obtenidas por inteligencia artificial y el segundo para el resto (Dialogos y noticias)

#### Frases obtenidas con inteligencia artificial

Se unen todos los archivos CSV en un solo DataFrame y añade una columna 'sarcasmos'.

In [2]:
def unir_frases_csv(directorio, es_sarcastico):
    
    # Crear una lista para almacenar los dataframes
    dataframes = []

    # Leer todos los archivos CSV que estén dentro del directorio
    for filename in os.listdir(directorio):
        if filename.endswith(".csv"):  # Solo selecciona los archivos CSV
            # Leer el CSV
            df = pd.read_csv(os.path.join(directorio, filename))
            # Añadir la columna 'sarcasmos' con el valor correspondiente
            df["sarcasmos"] = es_sarcastico
            # Cambiar el nombre de la columna existente a 'texto'
            df.columns = ["Texto", "Sarcasmo"]
            # Añadir el dataframe a la lista
            dataframes.append(df)

    # Unir todos los dataframes en uno solo
    df_final = pd.concat(dataframes, ignore_index=True)
    
    return df_final

df_no_sarcasticas = unir_frases_csv("Dialogos\\Frases_no_sarcasticas", 0)
print(df_no_sarcasticas.head())
df_sarcasticas = unir_frases_csv("Dialogos\\Frases_sarcasticas", 1)
print(df_sarcasticas.head())

                                               Texto  Sarcasmo
0  El fin de semana pasado salí con mis amigos y ...         0
1  Me encanta reunirme con mis amigos para ponern...         0
2  Este sábado tengo una quedada con amigos y est...         0
3  Siempre que nos reunimos con mis amigos, las c...         0
4  Anoche salimos a cenar con mis amigos y disfru...         0
                                               Texto  Sarcasmo
0  Claro, todos llegarán a la hora acordada... se...         1
1  Nada como organizar una cena y que la mitad ca...         1
2  ¡Oh, genial! Todos olvidaron traer lo que prom...         1
3  Qué suerte, el único día que puedo, nadie más ...         1
4  Ah, claro, porque siempre es fácil ponerse de ...         1


#### Diálogos y noticias

In [3]:
import os
import pandas as pd

def unir_csv(directorio):
    dataframes = []
    
    for filename in os.listdir(directorio):
        # Leer el CSV
        df = pd.read_csv(os.path.join(directorio, filename), sep=';')
        
        # Verificar cuántas columnas tiene el DataFrame
        if len(df.columns) == 3:
            # Si tiene 3 columnas, solo usar "Frase Actual" y "Sarcasmo"
            df = df[['Frase Actual', 'Sarcasmo']]
            df.columns = ["Texto", "Sarcasmo"]  # Renombrar columnas a "texto" y "sarcasmo"
        elif len(df.columns) == 2:
            # Si tiene 2 columnas, renombrar a "texto" y "sarcasmo"
            df.columns = ["Texto", "Sarcasmo"]
        
        # Añadir el DataFrame procesado a la lista
        dataframes.append(df)
    
    # Concatenar todos los DataFrames
    df_final = pd.concat(dataframes, ignore_index=True)
    
    return df_final

# Llamada a la función
df_procesado = unir_csv("Dialogos/Dialogos procesados")

Se concatenan los dataframes

In [4]:
dataframes = [df_no_sarcasticas, df_sarcasticas, df_procesado]

df = pd.concat(dataframes, ignore_index=True)

Se eliminan duplicados y valores nulos.

In [5]:
df = df.drop_duplicates()
df = df.dropna()

Se normalizan los valores de la columna sarcasmo, se reemplazan valores no válidos por 0 o 1 y y se convierte todo de nuevo a enteros.

In [ ]:
df.loc[:, 'Sarcasmo'] = df['Sarcasmo'].astype(str) 
df.loc[:, 'Sarcasmo'] = df['Sarcasmo'].replace({
    '0.': '0', 
    '=': '0', 
    'e0': '0', 
    '1.': '1',
    '0': '0', 
    '0 ': '0', 
    '1': '1', 
    '1 ': '1'
})
df.loc[:, 'Sarcasmo'] = df['Sarcasmo'].astype(int)

Se filtra el DataFrame para eliminar filas donde la columna 'Texto' contenga la palabra 'Horóscopo'

In [7]:
df = df[~df['Texto'].str.contains("Horóscopo", case=False, na=False)]

Se divide el DataFrame en dos partes: train y validation y se guardan los archivos asegurando que los textos estén entrecomillados si tienen comas.

In [8]:
train_df, validation_df = train_test_split(df, test_size=0.2, random_state=42)

train_df.to_csv('train.csv', index=False, sep=',', quoting=1)
validation_df.to_csv('validation.csv', index=False, sep=',', quoting=1)

Se guarda el CSV en el equipo

In [9]:
df.to_csv('Texto_procesado.csv', index=False,  sep=',', quoting=1)

In [10]:
df

,Texto,Sarcasmo
0,El fin de semana pasado salí con mis amigos y ...,0
1,Me encanta reunirme con mis amigos para ponern...,0
2,Este sábado tengo una quedada con amigos y est...,0
3,"Siempre que nos reunimos con mis amigos, las c...",0
4,Anoche salimos a cenar con mis amigos y disfru...,0
...,...,...
24294,Un niño de siete años empieza a sospechar que ...,1
24295,Una señora cree realmente que su hija podría i...,1
24296,Miles de palomas españolas emigran a Alemania ...,1
24297,El Real Madrid ficha por error al actor Christ...,1
